In [5]:
import numpy as np


def lennard_jones_potential(distance):
    if distance == 0:
        return float("-inf")
    return 1 / distance**12 - 2 / distance**6


def calculate_gradients(positions, num_atoms):
    gradients = np.zeros_like(positions)

    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                continue
            r_vec = positions[3 * i : 3 * i + 3] - positions[3 * j : 3 * j + 3]
            r_norm = np.linalg.norm(r_vec)
            force_direction = r_vec / r_norm
            force_magnitude = 12 * (1 / r_norm**13 - 1 / r_norm**7)
            gradients[3 * i : 3 * i + 3] += force_magnitude * force_direction

    return gradients


def calculate_hessian(positions, num_atoms):
    dim = len(positions)
    hessian = np.zeros((dim, dim))

    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                continue
            r_vec = positions[3 * i : 3 * i + 3] - positions[3 * j : 3 * j + 3]
            r_norm = np.linalg.norm(r_vec)
            unit_r_vec = r_vec / r_norm

            hess_ij = np.outer(unit_r_vec, unit_r_vec) * (
                156 / r_norm**14 - 42 / r_norm**8
            )
            hess_ij += np.eye(3) * (12 / r_norm**8 - 12 / r_norm**14)

            hessian[3 * i : 3 * i + 3, 3 * i : 3 * i + 3] += hess_ij
            hessian[3 * i : 3 * i + 3, 3 * j : 3 * j + 3] -= hess_ij

    return hessian


def backtracking_line_search(
    positions, gradients, hessian_inv, alpha=1.0, beta=0.8, c=0.0001
):
    while True:
        new_positions = positions + alpha * np.dot(hessian_inv, -gradients)
        if calculate_total_energy(new_positions) <= calculate_total_energy(
            positions
        ) + c * alpha * np.dot(gradients, np.dot(hessian_inv, -gradients)):
            break
        alpha *= beta
    return alpha


def newtons_method(positions, num_atoms, max_iterations=1000, tolerance=1e-10):
    for _ in range(max_iterations):
        gradients = calculate_gradients(positions, num_atoms)
        hessian = calculate_hessian(positions, num_atoms)

        try:
            hessian_inv = np.linalg.inv(hessian)
        except np.linalg.LinAlgError:
            print("Hessian is singular or near-singular")
            break

        step_size = backtracking_line_search(positions, gradients, hessian_inv)
        delta_positions = -step_size * np.dot(hessian_inv, gradients)
        positions += delta_positions

        if np.linalg.norm(gradients) < tolerance:
            break

    return positions

In [6]:
N = 3
init = np.random.rand(3*(N-1))   

optimum = newtons_method(init, N - 1)

print("Optimized positions:", optimum)


Hessian is singular or near-singular
Optimized positions: [0.40672074 0.39819823 0.20014955 0.59146637 0.5564027  0.73695577]
